# Семантическая Сегментация. Часть 3.

Датасет COCO - command objects in context.

В ноутбуке пример создания пайплайна данных для датасета COCO. Train датасет весит 18Gb.

Не всегда Google Colab предоставляет достаточно места для скачивания датасета. Возможно придётся скачивать датасет на локальную машину и подготовить локально (например отобрать картинки только с определённым классом, например с классом "человек"), и подготовленный локально датасет уже загрузить в колаб.

## Переключение версии TensorFlow

In [0]:
%tensorflow_version 2.x

In [0]:
import os
import skimage.io as io
import numpy as np

import tensorflow as tf

## Загрузка датасета COCO и COCO API

In [0]:
if 1:
    !mkdir -p data

    # !cd data && wget http://images.cocodataset.org/zips/train2017.zip 
    # !cd data && wget http://images.cocodataset.org/zips/val2017.zip 
    !cd data && wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip 

    # !cd data && unzip -q train2017.zip
    # !cd data && unzip -q val2017.zip
    !cd data && unzip -q annotations_trainval2017.zip

    # библиотека для работы с COCO API
    !cd data && git clone https://github.com/cocodataset/cocoapi
    !cd data/cocoapi/PythonAPI && make

## Подготовка COCO API

Импортируем библиотеку.

In [0]:
COCO_ROOT = './data/'
import sys
sys.path.insert(0, os.path.join(COCO_ROOT, 'cocoapi/PythonAPI'))
from pycocotools.coco import COCO

## Универсальный класс Dataset для сегментации

In [0]:
class Dataset():

    def crop_images(self, img, inp_size, random_crop=False):
        shape = tf.shape(img)
        pad = (
            [0, tf.maximum(inp_size - shape[0], 0)],
            [0, tf.maximum(inp_size - shape[1], 0)],
            [0, 0],
        )
        img = tf.pad(img, pad)

        if random_crop:
            img = tf.image.random_crop(img, (inp_size, inp_size, shape[2]))
        else: # central crop
            shape = tf.shape(img)
            ho = (shape[0] - inp_size) // 2
            wo = (shape[1] - inp_size) // 2
            img = img[ho:ho+inp_size, wo:wo+inp_size, :]

        return img

    def train_dataset(self, batch_size, epochs, inp_size):
    # картинки разного размера. Чтобы привести их к одному размеру будем делать кроп
    # кроп будем производить случайным образом, получится аугументация, что хорошо
        def item_to_images(item):
            # item - путь к картинке
            random_crop = True
            # read_images заточена под COCO, читает картинки, передаём путь
            # на выходе получаем комбинированную картинку, где по канальному измерению сконкатенирована
            # поканальная картинка RGB и картинка с метками классов (картинки будут иметь тип tf.uint8)
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            # вызываем кроп не заботясь о том, чтобы соответствовали исходная картинка и ground truth,
            # потому что получается 4-х канальная картинка
            # если random_crop, то берём случайный кроп с фиксированным размером, иначе центральный
            img_combined = self.crop_images(img_combined, inp_size, random_crop)
            
            # получаем картинку и маску (карту сегментации - ground truth)
            # исходная картинка в первых 3-х каналах img_combined, преобразуем во float32 и делим на 255
            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            # маску (сегментацию) извлекаем из последнего канала img_combined, который переводим в float32
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class

        # датасет - это список путей к файлам с картинками
        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        # перемешиваем пути. you have a dataset: [1, 2, 3, 4, 5, 6], then .shuffle(buffer_size=3) will allocate
        # a buffer of size 3 for picking random entries. This buffer will be connected to the source dataset
        # https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset
        dataset = dataset.shuffle(buffer_size=len(self.img_list))
        # для каждого элемента датасета применяем функцию item_to_images и получаем выход для каждого элемента
        # т.е. она по item читает картинку с диска и отдаёт изображение и карту сегментации
        dataset = dataset.map(item_to_images)
        # повторяем столько раз сколько эпох. As soon as all the entries are read from the dataset and you try
        # to read the next element, the dataset will throw an error. That's where ds.repeat() comes into play.
        # It will re-initialize the dataset, making it again like this: [1,2,3] <= [4,5,6]
        dataset = dataset.repeat(epochs)
        # The .batch() will take first batch_size entries and make a batch out of them. So, batch size of 3 for
        # our example dataset will produce two batch records:[2,1,5][3,6,4]
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset

    def val_dataset(self, batch_size, inp_size):
        # на валидационном датасети нельзя делать случайный кроп, поэтому здесь всегда
        # будет кроп из центра: random_crop = False и мы не делаем repeat по эпохам
        # просто датасет как есть

        def item_to_images(item):
            random_crop = False
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            img_combined = self.crop_images(img_combined, inp_size, random_crop)

            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class

        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        dataset = dataset.map(item_to_images)
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset

## Класс для сегментационного датасета COCO

Класс наследутся от универсального `Dataset` и реализует кастомную функцию чтения данных. Здесь специфичная для COCO логика.

In [0]:
class COCO_Dataset(Dataset):

    def __init__(self, sublist):
        # путь к разметке
        ann_file_fpath = os.path.join(COCO_ROOT, 'annotations', 'instances_'+sublist+'2017.json')
        # инициализируем класс COCO из COCO API
        self.coco = COCO(ann_file_fpath)
        # указываем какие категории мы хоти указывать (можно несколько в списке)
        self.cat_ids = self.coco.getCatIds(catNms=['person'])
        # с помощью COCO API получаем список картинок с категориями
        self.img_list = self.coco.getImgIds(catIds=self.cat_ids)

    def read_images(self, img_id):
        # функция для загрузки картинок по img_id (элементу списка img_list)
        img_id = int(img_id.numpy())
        # получаем картинку по img_id
        img_data = self.coco.loadImgs(img_id)[0]
        img_fname = '/'.join(img_data['coco_url'].split('/')[-2:])

        # логика по загрузке масок
        # в аннтоации может быть несколько масок по числу людей и каждая маска сидит в отдельном слое
        # плюсуем все маски в одну
        img = io.imread(os.path.join(COCO_ROOT, img_fname))
        if len(img.shape) == 2:
            img = np.tile(img[..., None], (1, 1, 3))

        ann_ids = self.coco.getAnnIds(imgIds=img_data['id'], catIds=self.cat_ids, iscrowd=None)
        anns = self.coco.loadAnns(ann_ids)
        mask_class = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        for i in range(len(anns)):
            mask_class += self.coco.annToMask(anns[i])
        mask_class = (mask_class > 0).astype(np.uint8)
        # получаем бинарную маску человек/фон даже если несколько людей

        # получаем картинку вместе с маской сегментации
        img_combined = np.concatenate([img, mask_class[..., None]], axis=2)

        return img_combined

In [0]:
# создаём train и валидационный датасеты
# строка train и val передаётся в COCO Dataset конструктор для чтения нужного json-файла
COCO_dataset_train = COCO_Dataset('train')
COCO_dataset_val = COCO_Dataset('val')

In [0]:
# из COCO датасетов получаем датасеты для tensorflow
# train_ds = COCO_dataset_train.train_dataset(...)
# val_ds = COCO_dataset_val.val_dataset(...)